In [48]:
import hashlib
import requests

import json
import os
 
# Refresh the MD5s in the ota_list.json file

with open('ota_list.json', 'r') as f:
    ota_list = json.load(f)

for device in ota_list.keys():

    for k in ota_list[device].keys():
        if k.startswith('ota'):
            url = ota_list[device][k]['url']
            dest = ota_list[device][k]['destination']
            file = requests.get(url).content
            md5 = hashlib.md5(file)
            ota_list[device][k]['md5'] = md5.hexdigest()

with open('ota_list.json', 'w') as f:
    json.dump(ota_list, f, indent=4)

        


In [50]:
# Example of how the microcontroller would check for updates

update_keys = []
device = "2b41"
localpath = "/Users/calum/git"

with open('ota_list.json', 'r') as f:
    ota_list = json.load(f)[device]

# Check MD5s of existing files
for k in ota_list.keys():
    if k.startswith('ota'):
        md5_ref = ota_list[k]['md5']
        dest = localpath+ota_list[k]['destination']
        with open(dest, 'rb') as f:
            file = f.read()
        md5 = hashlib.md5(file)
        if (md5_ref != md5.hexdigest()):
            print(f"MD5 mismatch for {dest}, adding to update list")
            update_keys.append(k)

if update_keys == []:
    print("No files need updating")
else:
    print("Files need updating")
    print(update_keys)

# Download the files to temporary locations
for k in update_keys:
    print(k)
    url = ota_list[k]['url']
    dest = localpath+ota_list[k]['destination']
    dest_temp = dest+'.ota_temp'
    file = requests.get(url).content
    with open(dest_temp, 'wb') as f:
        f.write(file)

# Confirm MD5s match
for k in update_keys:
    md5_ref = ota_list[k]['md5']
    dest = localpath+ota_list[k]['destination']
    dest_temp = dest+'.ota_temp'
    with open(dest_temp, 'rb') as f:
        file = f.read()
    md5 = hashlib.md5(file)
    if (md5_ref == md5.hexdigest()):
        print(f"MD5 match for {dest}")
    else: 
        print(f"MD5 mismatch for {dest}")
        raise Exception(f"MD5 mismatch for {dest}")
    # Log exception and reboot normally with original files

# Overwrite the original files with the OTA temp files
for k in update_keys:
    dest = localpath+ota_list[k]['destination']
    dest_temp = dest+'.ota_temp'
    with open(dest_temp, 'rb') as f:
        file = f.read()
    with open(dest, 'wb') as f:
        f.write(file)
        print(f"updated {dest}")
    os.remove(dest_temp)


MD5 mismatch for /Users/calum/git/circuitpy_mcu/mcu.py, adding to update list
Files need updating
['ota1']
ota1
MD5 match for /Users/calum/git/circuitpy_mcu/mcu.py
updated /Users/calum/git/circuitpy_mcu/mcu.py
